In [1]:
!pip install pmdarima

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [5]:
%matplotlib inline

import numpy as np
from numpy import concatenate, percentile
import pandas as pd
from pandas import read_csv, DataFrame, concat

import matplotlib.pyplot as plt
import xgboost as xgb

from math import sqrt
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

from matplotlib import pyplot

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from pmdarima.arima import auto_arima


In [6]:
#Set parameters to see all data
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [9]:
# Load Train dataset
train = pd.read_csv("../ds_takehome_task/train.csv")
train.head(5).append(train.tail(5))

/var/folders/7p/k227m_450jsfhrwrkgf7wdtw0000gn/T/ipykernel_55531/3869612840.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train.head(5).append(train.tail(5))


,productsGroup_key,date_key,quantitySales
0,1,20190902,26784.0
1,1,20190903,7432.0
2,1,20190904,1424.0
3,1,20190905,608.0
4,1,20190906,776.0
242395,297,20211123,18.0
242396,297,20211124,16.0
242397,297,20211125,16.0
242398,297,20211126,16.0
242399,297,20211127,38.0


In [10]:
# Load submission dataset
submission = pd.read_csv("../ds_takehome_task/submission.csv")
submission.head(5).append(submission.tail(5))

/var/folders/7p/k227m_450jsfhrwrkgf7wdtw0000gn/T/ipykernel_55531/1180049511.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission.head(5).append(submission.tail(5))


,productsGroup_key,date_key,quantitySales,q_0850,q_0900,q_0920,q_0950,q_0990
0,1,20211128,0,0,0,0,0,0
1,1,20211129,0,0,0,0,0,0
2,1,20211130,0,0,0,0,0,0
3,1,20211201,0,0,0,0,0,0
4,1,20211202,0,0,0,0,0,0
8311,297,20211221,0,0,0,0,0,0
8312,297,20211222,0,0,0,0,0,0
8313,297,20211223,0,0,0,0,0,0
8314,297,20211224,0,0,0,0,0,0
8315,297,20211225,0,0,0,0,0,0


In [11]:
train['isStationary'] = 0
for product in train.productsGroup_key.unique():
  series = train[train.productsGroup_key == product].quantitySales
  X = series.values
  result = adfuller(X)
  if (result[1] > 0.05):
    train.loc[train['productsGroup_key'] == product, 'isStationary'] = 1

In [12]:
train['isStationary'].value_counts()

0    200862
1     41538
Name: isStationary, dtype: int64

In [13]:
stationary_train = train[train['isStationary'] == 1].copy()
non_stationary_train = train[train['isStationary'] == 0].copy()

In [14]:
def outlier_detection(arr, isStationary):
  q25, q75 = percentile(arr, 15), percentile(arr, 85)
  iqr = q75 - q25
  if (isStationary == 1):
    # calculate the outlier cutoff
    cut_off = iqr * 1.5
  else:
    # calculate the outlier cutoff
    cut_off = iqr * 0.5
  lower, upper = q25 - cut_off, q75 + cut_off
  return lower, upper

In [15]:
for key in stationary_train['productsGroup_key'].unique():
  lower_st, upper_st = outlier_detection(stationary_train[stationary_train['productsGroup_key'] == key]['quantitySales'], 1)

  stationary_train.loc[stationary_train['productsGroup_key'] == key, 'lower'] = lower_st
  stationary_train.loc[stationary_train['productsGroup_key'] == key, 'upper'] = upper_st

In [16]:
for key in non_stationary_train['productsGroup_key'].unique():
  lower_nst, upper_nst = outlier_detection(non_stationary_train[non_stationary_train['productsGroup_key'] == key]['quantitySales'], 0)

  non_stationary_train.loc[non_stationary_train['productsGroup_key'] == key, 'lower'] = lower_nst
  non_stationary_train.loc[non_stationary_train['productsGroup_key'] == key, 'upper'] = upper_nst

In [17]:
stationary_train.iloc[1]

productsGroup_key          60.00
date_key             20190902.00
quantitySales            2331.00
isStationary                1.00
lower                   -8474.85
upper                   14525.55
Name: 48315, dtype: float64

In [18]:
stationary_train.loc[stationary_train['quantitySales'] < stationary_train['lower'], 'quantitySales'] = stationary_train['lower']
stationary_train.loc[stationary_train['quantitySales'] > stationary_train['upper'], 'quantitySales'] = stationary_train['upper']
non_stationary_train.loc[non_stationary_train['quantitySales'] < non_stationary_train['lower'], 'quantitySales'] = non_stationary_train['lower']
non_stationary_train.loc[non_stationary_train['quantitySales'] > non_stationary_train['upper'], 'quantitySales'] = non_stationary_train['upper']


In [19]:
stationary_train.shape, non_stationary_train.shape

((41538, 6), (200862, 6))

In [20]:
train_df = stationary_train.append(non_stationary_train)
train_df.shape

/var/folders/7p/k227m_450jsfhrwrkgf7wdtw0000gn/T/ipykernel_55531/2332453691.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = stationary_train.append(non_stationary_train)


(242400, 6)

In [21]:
train_df.head()

,productsGroup_key,date_key,quantitySales,isStationary,lower,upper
48314,60,20190901,153.0,1,-8474.85,14525.55
48315,60,20190902,2331.0,1,-8474.85,14525.55
48316,60,20190903,2781.0,1,-8474.85,14525.55
48317,60,20190904,3744.0,1,-8474.85,14525.55
48318,60,20190905,3078.0,1,-8474.85,14525.55


In [22]:
train_df.index = pd.to_datetime(train_df.date_key)
train_df.head()

,productsGroup_key,date_key,quantitySales,isStationary,lower,upper
date_key,,,,,,
1970-01-01 00:00:00.020190901,60,20190901,153.0,1,-8474.85,14525.55
1970-01-01 00:00:00.020190902,60,20190902,2331.0,1,-8474.85,14525.55
1970-01-01 00:00:00.020190903,60,20190903,2781.0,1,-8474.85,14525.55
1970-01-01 00:00:00.020190904,60,20190904,3744.0,1,-8474.85,14525.55
1970-01-01 00:00:00.020190905,60,20190905,3078.0,1,-8474.85,14525.55


In [23]:
train_df2 = train_df.copy()
train_df2 = train_df2[train_df2.productsGroup_key == 1]
train_df2.shape

(818, 6)

In [24]:
result_df = submission.copy()
result_df.head()

,productsGroup_key,date_key,quantitySales,q_0850,q_0900,q_0920,q_0950,q_0990
0,1,20211128,0,0,0,0,0,0
1,1,20211129,0,0,0,0,0,0
2,1,20211130,0,0,0,0,0,0
3,1,20211201,0,0,0,0,0,0
4,1,20211202,0,0,0,0,0,0


In [25]:
key_list = train_df.productsGroup_key.unique()
key_list.sort()


In [28]:
for key in key_list:
    train_df2 = train_df.copy()
    train_df2 = train_df2[train_df2.productsGroup_key == key]

    train_df_arima = train_df2.drop(columns = ['productsGroup_key', 'date_key', 'isStationary', 'lower', 'upper'], axis=1)


    # Arima Training
    now = datetime.now()
    print("Arima process started at : ",now)
    arima_stepwise_model = auto_arima(train_df_arima, start_p=0, start_q=0,
                           max_p=7, max_q=7, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True,
                           n_jobs=-1)


    end = datetime.now()
    print("Arima process finished at : ", end)
    print("Process took : ", end-now)

    print(arima_stepwise_model)

    # Fit arima model
    arima_predicts = arima_stepwise_model.fit(train_df_arima)

    # Predict results with auto-arima
    future_forecast = arima_predicts.predict(n_periods=28)

    stationary_train.loc[stationary_train['quantitySales'] < stationary_train['lower'], 'quantitySales'] = stationary_train['lower']

    result_df.loc[result_df['productsGroup_key'] == key, 'quantitySales'] = future_forecast

    result_df.to_csv("submission2.csv")

    print(key, " is written to csv")


Arima process started at :  2022-05-20 05:05:42.822161
Performing stepwise search to minimize aic


/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/pmdarima/arima/_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '


 ARIMA(0,1,0)(0,1,1)[7]             : AIC=inf, Time=0.36 sec
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=16473.990, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=15992.821, Time=0.24 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=0.57 sec
 ARIMA(1,1,0)(0,1,0)[7]             : AIC=16288.203, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[7]             : AIC=15994.012, Time=0.21 sec
 ARIMA(1,1,0)(1,1,1)[7]             : AIC=inf, Time=0.59 sec
 ARIMA(1,1,0)(0,1,1)[7]             : AIC=inf, Time=0.47 sec
 ARIMA(1,1,0)(2,1,1)[7]             : AIC=inf, Time=1.69 sec
 ARIMA(0,1,0)(1,1,0)[7]             : AIC=16207.911, Time=0.05 sec
 ARIMA(2,1,0)(1,1,0)[7]             : AIC=15947.755, Time=0.13 sec
 ARIMA(2,1,0)(0,1,0)[7]             : AIC=16188.077, Time=0.04 sec
 ARIMA(2,1,0)(2,1,0)[7]             : AIC=15898.258, Time=0.21 sec
 ARIMA(2,1,0)(2,1,1)[7]             : AIC=inf, Time=1.46 sec
 ARIMA(2,1,0)(1,1,1)[7]             : AIC=inf, Time=0.86 sec
 ARIMA(3,1,0)(2,1,0)[7]             :

/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/pmdarima/arima/_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '


 ARIMA(0,1,0)(0,1,1)[7]             : AIC=14280.522, Time=0.33 sec
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=14672.232, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=14267.474, Time=0.20 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=13821.064, Time=0.49 sec
 ARIMA(0,1,1)(0,1,0)[7]             : AIC=inf, Time=0.06 sec
 ARIMA(0,1,1)(1,1,1)[7]             : AIC=13817.300, Time=0.76 sec
 ARIMA(0,1,1)(1,1,0)[7]             : AIC=inf, Time=0.20 sec
 ARIMA(0,1,1)(2,1,1)[7]             : AIC=13818.372, Time=0.97 sec
 ARIMA(0,1,1)(1,1,2)[7]             : AIC=inf, Time=1.44 sec
 ARIMA(0,1,1)(0,1,2)[7]             : AIC=13816.669, Time=1.21 sec
 ARIMA(0,1,0)(0,1,2)[7]             : AIC=14276.922, Time=0.66 sec
 ARIMA(1,1,1)(0,1,2)[7]             : AIC=13818.638, Time=1.17 sec
 ARIMA(0,1,2)(0,1,2)[7]             : AIC=13818.638, Time=1.82 sec
 ARIMA(1,1,0)(0,1,2)[7]             : AIC=14051.929, Time=1.11 sec
 ARIMA(1,1,2)(0,1,2)[7]             : AIC=inf, Time=2.10 sec
 ARIMA(0,1,1)(0,1,2

...

 ARIMA(0,1,0)(0,1,1)[7]             : AIC=inf, Time=0.47 sec
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=16521.573, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=16125.480, Time=0.06 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=0.53 sec
 ARIMA(1,1,0)(0,1,0)[7]             : AIC=16304.121, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[7]             : AIC=16034.968, Time=0.58 sec
 ARIMA(1,1,0)(2,1,1)[7]             : AIC=inf, Time=1.51 sec
 ARIMA(1,1,0)(1,1,1)[7]             : AIC=inf, Time=0.56 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=16244.145, Time=0.10 sec
 ARIMA(2,1,0)(2,1,0)[7]             : AIC=16000.387, Time=0.21 sec
 ARIMA(2,1,0)(1,1,0)[7]             : AIC=16054.129, Time=0.11 sec
 ARIMA(2,1,0)(2,1,1)[7]             : AIC=inf, Time=0.70 sec
 ARIMA(2,1,0)(1,1,1)[7]             : AIC=inf, Time=0.75 sec
 ARIMA(3,1,0)(2,1,0)[7]             : AIC=15950.285, Time=0.33 sec
 ARIMA(3,1,0)(1,1,0)[7]             : AIC=16007.703, Time=0.14 sec
 ARIMA(3,1,0)(2,1,1)[7]        

/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/pmdarima/arima/_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '


 ARIMA(0,1,0)(0,1,1)[7]             : AIC=inf, Time=0.27 sec
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=14376.955, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=13916.493, Time=0.20 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=0.51 sec
 ARIMA(1,1,0)(0,1,0)[7]             : AIC=14220.662, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[7]             : AIC=13847.275, Time=0.54 sec
 ARIMA(1,1,0)(2,1,1)[7]             : AIC=inf, Time=1.20 sec
 ARIMA(1,1,0)(1,1,1)[7]             : AIC=inf, Time=0.50 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=14005.914, Time=0.10 sec
 ARIMA(2,1,0)(2,1,0)[7]             : AIC=13763.595, Time=0.65 sec
 ARIMA(2,1,0)(1,1,0)[7]             : AIC=13827.483, Time=0.34 sec
 ARIMA(2,1,0)(2,1,1)[7]             : AIC=inf, Time=1.14 sec
 ARIMA(2,1,0)(1,1,1)[7]             : AIC=inf, Time=0.73 sec
 ARIMA(3,1,0)(2,1,0)[7]             : AIC=13703.970, Time=0.97 sec
 ARIMA(3,1,0)(1,1,0)[7]             : AIC=13777.159, Time=0.42 sec
 ARIMA(3,1,0)(2,1,1)[7]        

/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/pmdarima/arima/_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '


 ARIMA(0,1,0)(0,1,1)[7]             : AIC=inf, Time=0.28 sec
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=8242.391, Time=0.01 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=7810.542, Time=0.06 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=inf, Time=0.35 sec
 ARIMA(1,1,0)(0,1,0)[7]             : AIC=8037.310, Time=0.02 sec
 ARIMA(1,1,0)(2,1,0)[7]             : AIC=7735.086, Time=0.16 sec
 ARIMA(1,1,0)(2,1,1)[7]             : AIC=inf, Time=1.36 sec
 ARIMA(1,1,0)(1,1,1)[7]             : AIC=inf, Time=0.55 sec
 ARIMA(0,1,0)(2,1,0)[7]             : AIC=7952.511, Time=0.10 sec
 ARIMA(2,1,0)(2,1,0)[7]             : AIC=7647.875, Time=0.19 sec
 ARIMA(2,1,0)(1,1,0)[7]             : AIC=7710.663, Time=0.09 sec
 ARIMA(2,1,0)(2,1,1)[7]             : AIC=inf, Time=1.44 sec
 ARIMA(2,1,0)(1,1,1)[7]             : AIC=inf, Time=0.82 sec
 ARIMA(3,1,0)(2,1,0)[7]             : AIC=7603.913, Time=0.23 sec
 ARIMA(3,1,0)(1,1,0)[7]             : AIC=7663.213, Time=0.12 sec
 ARIMA(3,1,0)(2,1,1)[7]             : AI

In [41]:
submission_df = pd.read_csv("../ds_takehome_task/submission2.csv")
submission_df.head()

,Unnamed: 0,productsGroup_key,date_key,quantitySales,q_0850,q_0900,q_0920,q_0950,q_0990
0,0,1,20211128,84.393529,0,0,0,0,0
1,1,1,20211129,6003.109275,0,0,0,0,0
2,2,1,20211130,9846.854537,0,0,0,0,0
3,3,1,20211201,7299.192248,0,0,0,0,0
4,4,1,20211202,2445.539404,0,0,0,0,0


In [42]:
submission_df.dtypes

Unnamed: 0             int64
productsGroup_key      int64
date_key               int64
quantitySales        float64
q_0850                 int64
q_0900                 int64
q_0920                 int64
q_0950                 int64
q_0990                 int64
dtype: object

In [43]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [44]:
submission_df = reduce_mem_usage(submission_df)

Mem. usage decreased to  0.13 Mb (76.4% reduction)


In [45]:
submission_df.dtypes

Unnamed: 0             int16
productsGroup_key      int16
date_key               int32
quantitySales        float32
q_0850                  int8
q_0900                  int8
q_0920                  int8
q_0950                  int8
q_0990                  int8
dtype: object

In [48]:
submission_df['quantitySales'] = round(submission_df['quantitySales'])
submission_df.head()

,Unnamed: 0,productsGroup_key,date_key,quantitySales,q_0850,q_0900,q_0920,q_0950,q_0990
0,0,1,20211128,84.0,0,0,0,0,0
1,1,1,20211129,6003.0,0,0,0,0,0
2,2,1,20211130,9847.0,0,0,0,0,0
3,3,1,20211201,7299.0,0,0,0,0,0
4,4,1,20211202,2446.0,0,0,0,0,0


In [62]:
import sqlite3 as sql


In [64]:
conn = sql.connect("../ds_takehome_task/submission.db")
submission_df.to_sql("../ds_takehome_task/submission", conn, if_exists='replace')

8316

# **AutoArima**

In [21]:
train_df2[:-28].head(5)

,productsGroup_key,date_key,quantitySales,isStationary,lower,upper
date_key,,,,,,
1970-01-01 00:00:00.020190902,1,20190902,11794.4,0,-3824.8,11794.4
1970-01-01 00:00:00.020190903,1,20190903,7432.0,0,-3824.8,11794.4
1970-01-01 00:00:00.020190904,1,20190904,1424.0,0,-3824.8,11794.4
1970-01-01 00:00:00.020190905,1,20190905,608.0,0,-3824.8,11794.4
1970-01-01 00:00:00.020190906,1,20190906,776.0,0,-3824.8,11794.4


In [22]:
train_df2.columns


Index(['productsGroup_key', 'date_key', 'quantitySales', 'isStationary', 'lower', 'upper'], dtype='object')

In [23]:
train_df_arima = train_df2.drop(columns = ['productsGroup_key', 'date_key', 'isStationary', 'lower', 'upper'], axis=1)
train_df_arima.head()

,quantitySales
date_key,
1970-01-01 00:00:00.020190902,11794.4
1970-01-01 00:00:00.020190903,7432.0
1970-01-01 00:00:00.020190904,1424.0
1970-01-01 00:00:00.020190905,608.0
1970-01-01 00:00:00.020190906,776.0


In [24]:
arima_train = train_df_arima[:-28]
arima_test = train[-28:]

In [78]:
# Arima Training
now = datetime.now()
print("Arima process started at : ",now)

arima_stepwise_model = auto_arima(train_df_arima, start_p=2, start_q=2,
                           max_p=12, max_q=12, m=12,
                           start_P=2, seasonal=True,
                           d=2, D=2, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True,
                           n_jobs=-1)#n_jobs for parallel process
end = datetime.now()
print("Arima process finished at : ", end)
print("Process took : ", end-now)

print(arima_stepwise_model)

Arima process started at :  2022-05-19 23:57:16.328975
Performing stepwise search to minimize aic


/usr/local/lib/python3.7/dist-packages/pmdarima/arima/_validation.py:77: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  'Falling back to stepwise parameter search.' % n_jobs)


 ARIMA(2,2,2)(2,2,1)[12]             : AIC=16502.441, Time=41.04 sec
 ARIMA(0,2,0)(0,2,0)[12]             : AIC=17951.715, Time=0.17 sec
 ARIMA(1,2,0)(1,2,0)[12]             : AIC=17295.871, Time=1.58 sec
 ARIMA(0,2,1)(0,2,1)[12]             : AIC=inf, Time=7.74 sec
 ARIMA(2,2,2)(1,2,1)[12]             : AIC=inf, Time=23.29 sec
 ARIMA(2,2,2)(2,2,0)[12]             : AIC=inf, Time=33.47 sec
 ARIMA(2,2,2)(2,2,2)[12]             : AIC=16496.280, Time=51.21 sec
 ARIMA(2,2,2)(1,2,2)[12]             : AIC=16497.472, Time=52.50 sec
 ARIMA(1,2,2)(2,2,2)[12]             : AIC=16545.796, Time=54.88 sec
 ARIMA(2,2,1)(2,2,2)[12]             : AIC=inf, Time=nan sec
 ARIMA(3,2,2)(2,2,2)[12]             : AIC=16464.354, Time=57.68 sec
 ARIMA(3,2,2)(1,2,2)[12]             : AIC=16493.918, Time=54.59 sec
 ARIMA(3,2,2)(2,2,1)[12]             : AIC=inf, Time=47.45 sec
 ARIMA(3,2,2)(1,2,1)[12]             : AIC=inf, Time=27.80 sec
 ARIMA(3,2,1)(2,2,2)[12]             : AIC=inf, Time=51.35 sec
 ARIMA(4,2,2

In [79]:
# Fit arima model
arima_predicts = arima_stepwise_model.fit(arima_train)

In [80]:
# Predict results with auto-arima
future_forecast = arima_predicts.predict(n_periods=28)

In [81]:
arima_rmse = sqrt(mean_squared_error(train_df_arima['quantitySales'][-28:], future_forecast))

In [82]:
def mean_absolute_percentage_error(y_true, y_pred):
  return np.mean(np.abs((np.array(y_true) - np.array(y_pred)) / y_true)) * 100

In [83]:
arima_mape = mean_absolute_percentage_error(train_df_arima['quantitySales'][-28:], future_forecast)

In [84]:
print("Test RMSE:", np.mean(arima_rmse))
print("Test MAPE:", np.mean(arima_mape))

Test RMSE: 5332.394691268515
Test MAPE: 1963.0870261759019


In [69]:
result_df = train_df[-28:]


In [36]:
result_df['Predictions'] = future_forecast
result_df.to_csv("result.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
result_df.head()

Index(['productsGroup_key', 'date_key', 'quantitySales', 'isStationary', 'lower', 'upper', 'XGB_Predictions', 'Arima_Predictions'], dtype='object')